Import libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [7]:
dataset = pd.read_csv("/content/water_potability.csv");
dataset.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


# Model

Find correlation

In [ ]:
plt.figure(figsize=[20,10])
heat = dataset.corr()
sns.heatmap(heat, cmap=['#FF87CA','#ff96d1','#ffb0dc','#ffcce8','white'], annot=True)
plt.title("Checking for Correlations", size=40, pad=20)
sns.despine()
plt.show()

Divide dataset

In [ ]:
x = dataset.drop(['Potability'], axis = 1)
y = dataset['Potability']

Use "SMOTE" to balance dataset

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size = 0.3, random_state = 42)

smote = SMOTE()

X_train_balanced, Y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
X_train_balanced.shape, Y_train_balanced.shape

Train the model

In [ ]:
model = DecisionTreeClassifier(max_depth = 3)

model.fit(X_train_balanced, Y_train_balanced)